In [9]:
import plotly.express as px
import dash
import pandas as pd
import mysql.connector 
from dotenv import load_dotenv
import os
import numpy as np
import plotly.graph_objects as go

load_dotenv()
# Variables
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
USER = os.getenv("USER_DB")

# DB connection
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database = "weatherData"
)

cursor = mydb.cursor()

sql = '''
SELECT 
	c.temp - 272.15 as TEMP,
  l.location_name,
  c.dt as timestamp,
  hp.temp - 272.15 as pred_TEMP,
  hp.dt as pred_timestamp,
  hp.feels_like - 272.15 as pred_feels_like
FROM weatherData.current c 
JOIN weatherData.locations l ON (l.id = c.id_location)
JOIN weatherData.hourly_pred hp ON (hp.id_current = c.id_current);
'''
cursor.execute(sql)

df_current = pd.DataFrame(cursor.fetchall())
df_current.columns = next(zip(*cursor.description))

df_current['timestamp'] = pd.to_datetime(df_current['timestamp'], unit='s')
df_current['pred_timestamp'] = pd.to_datetime(df_current['pred_timestamp'], unit='s') 


In [16]:
brno_data = df_current.loc[df_current['location_name']=='Brno']

fig = px.line(brno_data, x='timestamp', y='TEMP')
fig.add_scatter(x=brno_data['pred_timestamp'], y=brno_data['pred_feels_like'], mode='markers')
fig.show()

In [11]:
aggr_brno_data = brno_data.groupby('pred_timestamp').agg(
    min_temperature=('pred_TEMP', 'min'),
    max_temperature=('pred_TEMP', 'max'),
    avg_temperature=('pred_TEMP', 'mean')
).reset_index()

aggr_brno_data

,pred_timestamp,min_temperature,max_temperature,avg_temperature
0,2024-08-08 18:00:00,21.189996,21.189996,21.189996
1,2024-08-08 19:00:00,20.330011,20.659998,20.495004
2,2024-08-08 20:00:00,19.770013,19.899988,19.820001
3,2024-08-08 21:00:00,18.830011,19.480005,19.195001
4,2024-08-08 22:00:00,17.480005,18.890009,18.373999
...,...,...,...,...
325,2024-08-22 07:00:00,18.749994,19.149988,18.829993
326,2024-08-22 08:00:00,20.450006,21.059991,20.907495
327,2024-08-22 09:00:00,22.510004,22.770013,22.596674
328,2024-08-22 10:00:00,23.990015,24.490015,24.240015


In [4]:


fig = go.Figure()
fig.add_trace(go.Scatter(x=brno_data['timestamp'], y=brno_data['TEMP'],
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=aggr_brno_data['pred_timestamp'].to_list()+aggr_brno_data['pred_timestamp'].to_list()[::-1], y=aggr_brno_data['max_temperature'].to_list()+aggr_brno_data['min_temperature'].to_list()[::-1],
                    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='Fair'))


fig.show()

In [12]:
brno_data

,TEMP,location_name,timestamp,pred_TEMP,pred_timestamp,pred_feels_like
0,21.19,Brno,2024-08-08 18:11:39,21.189996,2024-08-08 18:00:00,21.459985
1,21.19,Brno,2024-08-08 18:11:39,20.659998,2024-08-08 19:00:00,20.929987
2,21.19,Brno,2024-08-08 18:11:39,19.770013,2024-08-08 20:00:00,19.999994
3,21.19,Brno,2024-08-08 18:11:39,18.830011,2024-08-08 21:00:00,19.020013
4,21.19,Brno,2024-08-08 18:11:39,17.480005,2024-08-08 22:00:00,17.589990
...,...,...,...,...,...,...
13339,28.21,Brno,2024-08-20 12:00:07,19.149988,2024-08-22 07:00:00,18.399988
13340,28.21,Brno,2024-08-20 12:00:07,20.450006,2024-08-22 08:00:00,19.760004
13341,28.21,Brno,2024-08-20 12:00:07,22.770013,2024-08-22 09:00:00,22.149988
13342,28.21,Brno,2024-08-20 12:00:07,24.490015,2024-08-22 10:00:00,23.890009
